In [1]:
import tensorflow as tf
from keras.callbacks import TensorBoard
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50

2024-05-21 16:52:52.922791: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-21 16:52:53.188110: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-21 16:52:54.040902: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [5]:
DATA_DIR = "/media/christian/USB STICK/Images/Subset"
seed = 10
image_width = 1920
image_height = 1090
im_shape = (image_width, image_height)
BATCH_SIZE = 4

In [3]:
data_generator = ImageDataGenerator(
        validation_split=0.2,
        # width_shift_range=0.2,
        # height_shift_range=0.2,
        rescale=1./255,
        # shear_range=0.2,
        zoom_range=0.1,
        rotation_range=20,
        horizontal_flip=True,
        fill_mode='nearest'
)

In [6]:
train_generator = data_generator.flow_from_directory(DATA_DIR, target_size=im_shape, shuffle=True, seed=seed,
                                                     class_mode='categorical', batch_size=BATCH_SIZE, subset="training")
validation_generator = data_generator.flow_from_directory(DATA_DIR, target_size=im_shape, shuffle=False, seed=seed,
                                                     class_mode='categorical', batch_size=BATCH_SIZE, subset="validation")


nb_train_samples = train_generator.samples
nb_validation_samples = validation_generator.samples
classes = list(train_generator.class_indices.keys())
print('Classes: '+str(classes))
num_classes  = len(classes)

Found 694 images belonging to 4 classes.
Found 172 images belonging to 4 classes.
Classes: ['0', '1', '2', '3']


In [ ]:
print(type(nb_train_samples))

In [7]:
from collections import Counter
counter = Counter(train_generator.classes)
max_val = float(max(counter.values()))
class_weights = {class_id : nb_train_samples/(num_images * num_classes) for class_id, num_images in counter.items()}
     

The code below defines the layers and the hyperparameters to be tuned, as well as the parameters for tuning the hyperparameters.

In [38]:
import keras_tuner as kt
tuner = kt.tuners.BayesianOptimization(
  kt.applications.HyperResNet(input_shape=(image_width,image_height,3), classes=4),
  objective='val_accuracy',
  max_trials=50)

In [34]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
#this will stop the training if the model's val_loss does not improve over 3
#consecutive epochs

In [39]:
tuner.search(train_generator, epochs=10, validation_data=validation_generator,
             callbacks=[TensorBoard("/tmp/tb_logs")])


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
v1                |v1                |version
4                 |4                 |conv3_depth
36                |36                |conv4_depth
avg               |avg               |pooling
sgd               |sgd               |optimizer
0.01              |0.01              |learning_rate

Epoch 1/10


KeyboardInterrupt: 

In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials=3)
print("Best run:")
print(best_hps[0].values)
print("2nd best run:")
print(best_hps[1].values)
print("3rd best run:")
print(best_hps[2].values)

In [ ]:
model = tuner.hypermodel.build(best_hps[0])

In [ ]:
history = model.fit(X_train, y_train, epochs=10, validation_split=0.2,
                    callbacks=[stop_early])

Best model: val_acc

In [ ]:
%load_ext tensorboard

%tensorboard --logdir /tmp/tb_logs

In [ ]:
!zip -r tensorboard_logs.zip /tmp/tb_logs